In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 11

In [4]:
name = list[k].getName()
name

'RP55'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=1)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsMC,resultsFORM, resultsSORM, resultFORMIS, resultSubset]

In [7]:
a = "[%.8f,%.8f] " % ( resultsMC[4], resultsMC[5])


b = "[%.8f,%.8f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.8f,%.8f] " % ( resultSubset[4], resultSubset[5])


In [8]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.45379403,0.67120597] ', None],
 [None, None, None, '[0.45379403,0.67120597] ', None],
 [None, None, None, '[0.45379403,0.67120597] ', None],
 [None, None, None, '[0.45379403,0.67120597] ', None],
 [None, None, None, '[0.45379403,0.67120597] ', None],
 [None, None, None, '[0.45379403,0.67120597] ', None]]

In [9]:
l = np.array([[1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1],[1, 1, 1, a, 1, 1]])

In [10]:

method = ["CMC", "FORM", "SORM", "FORM-IS", "SUBSET"]
metrics = ["Probabilité exacte de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  result[i][2]

results[0][3] = "-----------------"
results[1][3] = "-----------------"
results[2][3] = a

results[3][3] = c
 
results[4][3] = b


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x1,x2]->[var g1 := 0.2 + 0.6 * (x1 - x2)^4 - (x1 - x2) / sqrt(2);var g2 := 0.2 + 0.6 * (x1 - x2)^4 + (x1 - x2) / sqrt(2);var g3 := (x1 - x2) + 5 / sqrt(2) - 2.2;var g4 := (x2 - x1) + 5 / sqrt(2) - 2.2;gsys := min(g1, g2, g3, g4)]


,Probabilité exacte de RP55,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
CMC,5.925926e-01,5.625e-01,3.0092593e-02,-----------------,80,1.2942966130050133
FORM,5.925926e-01,3.990057e-01,1.9358687e-01,-----------------,36,0.48588031294652073
SORM,5.925926e-01,3.9262876e-01,1.9996384e-01,"[0.45379403,0.67120597]",49,0.4718047584029549
FORM-IS,5.925926e-01,5.3940636e-01,5.3186253e-02,"[0.55317803,0.57262197]",138,1.0469568182457865
SUBSET,5.925926e-01,5.629e-01,2.9692592e-02,"[0.43395017,0.64486251]",10000,1.3001080990585347


In [11]:
df.to_csv('problem55.csv')